# Emerging Technologies Problems

This notebook contains the solutions to the Emerging Technologies problems 1 - 5. It contains markdown cells explaining the code in code cells.

## Problem 1: Generating Random Boolean Functions

## 

## Problem 2: Classical Testing for Function Type

## 

## Problem 3: Quantum Oracles

## 

## Problem 4: Deutsch's Algorithm with Qiskit

## 

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm

## 